In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/content/clinicsales.csv')
df.describe()

clinicnumber   treatment    purchase  numdoctors  avgpanelsize  \
count    322.000000  322.000000  322.000000  322.000000    322.000000   
mean     161.500000    1.015528    0.276398    3.934783   2303.751553   
std       93.097619    0.822053    0.651862    1.170284    141.683691   
min        1.000000    0.000000    0.000000    1.000000   2050.000000   
25%       81.250000    0.000000    0.000000    3.000000   2186.000000   
50%      161.500000    1.000000    0.000000    4.000000   2308.000000   
75%      241.750000    2.000000    0.750000    5.000000   2419.750000   
max      322.000000    2.000000    9.000000    7.000000   2546.000000   

         distance  
count  322.000000  
mean    35.313665  
std     43.026230  
min      2.000000  
25%      9.000000  
50%     21.000000  
75%     45.750000  
max    437.000000

In [9]:
# Purchase should be dummy variable that only takes values of 0 or 1.
df = df[df['purchase']!=9]
df.describe()

clinicnumber   treatment    purchase  numdoctors  avgpanelsize  \
count    321.000000  321.000000  321.000000  321.000000    321.000000   
mean     161.732087    1.012461    0.249221    3.931464   2303.267913   
std       93.149627    0.821489    0.433237    1.170593    141.638437   
min        1.000000    0.000000    0.000000    1.000000   2050.000000   
25%       81.000000    0.000000    0.000000    3.000000   2186.000000   
50%      162.000000    1.000000    0.000000    4.000000   2308.000000   
75%      242.000000    2.000000    0.000000    5.000000   2419.000000   
max      322.000000    2.000000    1.000000    7.000000   2546.000000   

         distance  
count  321.000000  
mean    35.383178  
std     43.075293  
min      2.000000  
25%      9.000000  
50%     21.000000  
75%     46.000000  
max    437.000000

# 1. Evidence on the validity of the experiment / showing that the randomization was successful.

In [15]:
# An exhibit showing the means of all outcomes in each treatment arm.
print(df.mean())
df_treatment = df.groupby('treatment')[['numdoctors', 'avgpanelsize', 'distance']].mean()
df_treatment['size'] = df.groupby('treatment')['clinicnumber'].count()
print(df_treatment)

clinicnumber     161.732087
treatment          1.012461
purchase           0.249221
numdoctors         3.931464
avgpanelsize    2303.267913
distance          35.383178
dtype: float64
           numdoctors  avgpanelsize   distance  size
treatment                                           
0            3.754717   2291.792453  38.028302   106
1            3.980952   2282.619048  35.161905   105
2            4.054545   2334.036364  33.045455   110


The means across the treatment arms are quite similar, as are the sample sizes. The data appear to be balanced on observable characteristics, which is consistent with the expectations of successful randomization.

# 2. An exhibit showing the means of all outcomes in each treatment arm, as well as 95% confidence intervals for the means.

In [33]:
# 95% confidence intervals for the means.
df_ate = df.groupby('treatment')['purchase'].agg(
    mean = ('mean'), std = ('std'), N = ('size'), se_sem =('sem')
)
df_ate['se'] = df_ate['std'] / np.sqrt(df_ate['N'])
df_ate['lb'] = df_ate['mean'] - 1.96*df_ate['se']
df_ate['ub'] = df_ate['mean'] + 1.96*df_ate['se']
print(df_ate[['mean', 'lb', 'ub']])

               mean        lb        ub
treatment                              
0          0.122642  0.059898  0.185385
1          0.333333  0.242732  0.423934
2          0.290909  0.205644  0.376174


# 3. An exhibit showing the ATEs for each outcome, as well las 95% confidence intervals for the ATE.

In [39]:
# ATEs for each outcome.
ate_table = df_ate['mean'] - df_ate['mean'][0]
ate_table = ate_table.to_frame(name = 'ate') # Creates a DataFrame with the original data as one column.
print(ate_table)

                ate
treatment          
0          0.000000
1          0.210692
2          0.168268


In [40]:
# 95% confidence intervals for the ATE.
ate_table['se'] = np.sqrt(df_ate['se']**2 + df_ate['se'][0]**2)
ate_table['lb'] = ate_table['ate'] - 1.96*ate_table['se']
ate_table['ub'] = ate_table['ate'] + 1.96*ate_table['se']
print(ate_table[['ate', 'lb', 'ub']][1:4])

                ate        lb        ub
treatment                              
1          0.210692  0.100486  0.320898
2          0.168268  0.062405  0.274130


# 4. A brief (1-3 sentence is fine) discussion of the business implications of the experiment. What should we do with the findings.

The results indicate that the "easier to use" pitch (Treatment 1) is the most effective for converting clinics to purchase. EMS should standardize this pitch to improve sales efficiency and clinic partnerships while using these insights to train new sales representatives for better engagement and conversion.

# 5. A brief (1-3 sentence is fine) about possible issues. Is the outcome metric ideal, or what would be better? Did you spot anything troubling in the balance checks or results?

The confidence intervals for the treatment effects are wide and overlap, so we should be cautious about the estimated 21% sales increase and whether the "easier to use" pitch (Treatment 1) is truly better than the "fewer errors" pitch (Treatment 2). Both pitches may be equally effective. EMS could explore additional measures, such as total purchase amounts, follow-up meeting rates, call satisfaction, feedback, or return rates. Long-term sales trends and potential spillover effects on other products should also be evaluated.